In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
#定义LeNet5模型，模型计算过程参考：https://blog.csdn.net/liaomin416100569/article/details/130677530?spm=1001.2014.3001.5501
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            #C1 层（卷积层）：6@28×28 该层使用了 6 个卷积核，每个卷积核的大小为 5×5，这样就得到了 6 个 feature map（特征图）。
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(inplace=True),
            #S2 层（下采样层，也称池化层）：6@14×14,池化单元为 2×2，因此，6 个特征图的大小经池化后即变为 14×14
            nn.MaxPool2d(kernel_size=2, stride=2),
            #C3 层（卷积层）：16@10×10  C3 层有 16 个卷积核，卷积模板大小为 5×5 C3 层的特征图大小为（14-5+1）×（14-5+1）= 10×10。
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(inplace=True),
            #S4（下采样层，也称池化层）：16@5×5,与 S2 的分析类似，池化单元大小为 2×2，因此，该层与 C3 一样共有 16 个特征图，每个特征图的大小为 5×5。
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            #LeNet-5模型中的C5层是一个全连接层。在LeNet-5模型中，前两个卷积层（C1和C3）之后是一个池化层（S2和S4），
            # 然后是一个全连接层（C5），最后是输出层（F6）。全连接层C5的输入是S4层的输出，它将这个输入展平为一个向量，
            # 并将其连接到输出层F6。因此，C5层是一个全连接层，而不是卷积层，这里和文中有些冲突。
            #C5 层（卷积层）：120 该层有 120 个卷积核，每个卷积核的大小仍为 5×5，因此有 120 个特征图,特征图大小为（5-5+1）×（5-5+1）= 1×1。这样该层就刚好变成了全连接
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(inplace=True),
            #F6 层（全连接层）：84,该层有 84 个特征图，特征图大小与 C5 一样都是 1×1
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            # OUTPUT 层（输出层）：10
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.features(x)
        #张量x在第0个维度上的大小，因为第0个维度是数据批次数（行数），s4层后的维度是(批次数，16,5,5)
        #转换成2维就是(行数,16*5*5)，-1表示自动计算合并成最后一个维度
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
model = LeNet5()
"""
transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])是一种数据预处理操作，用于对图像数据进行归一化处理。这个操作会将每个像素的数值减去均值(mean)并除以标准差(std)。
在这个例子中，mean=[0.5, 0.5, 0.5]表示将每个通道的像素值减去0.5，std=[0.5, 0.5, 0.5]表示将每个通道的像素值除以0.5。这样处理后，图像的像素值会在-1到1之间。
归一化可以帮助提高模型的训练效果和稳定性，因为它可以使输入数据的分布更加接近标准正态分布。此外，对于不同的数据集，可能需要不同的均值和标准差进行归一化操作，以使数据的分布更加合理。
在使用PyTorch的transforms.Normalize时，通常需要将其与其他数据预处理操作一起使用，例如transforms.ToTensor()将图像转换为张量。可以通过transforms.Compose将多个预处理操作组合在一起，形成一个数据预处理管道。
"""
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
#下载训练集,data是数据数组，target是标签
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
#下载测试集
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
#数据批处理和打乱，一次64条数据
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
#使用交叉熵损失函数
criterion = nn.CrossEntropyLoss()
#使用随机梯度下降法优化参数，梯度下降的学习率是0.001
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#判断是否有gpu如果有的话讲模型附加到cuda设备上
#momentum参数通过累积之前的梯度信息，使得参数更新具有一定的惯性，从而在参数空间中更快地找到全局最优解或局部最优解。
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#模型对数据集进行10次epoch
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")
"""
model.eval()是PyTorch中用于将模型设置为评估模式的函数。当调用model.eval()时，模型的行为会发生变化，包括：
   1. Batch Normalization和Dropout等具有随机性的层会固定住，不再产生随机变化。
   2. 模型的参数不会被更新，即不会进行梯度计算和反向传播。
   3. 在推断阶段，模型会根据输入数据生成输出，而不会进行训练。
  通常，在测试或评估模型时，需要调用model.eval()来确保模型的行为与训练时保持一致。
这样可以避免由于Batch Normalization和Dropout等层的随机性而导致结果不稳定。在调用model.train()之前，
应该使用model.eval()将模型切换回训练模式,要将模型切换回训练模式，可以使用model.train()方法。
"""   
model.eval()
correct = 0
total = 0
#torch.no_grad()是一个上下文管理器，将其包裹的代码块中的所有操作都不会计算梯度。
# 通常用于在不需要计算梯度的情况下进行推理或评估。
with torch.no_grad():
    for images, labels in test_loader:
        #数据加载到显存
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        #获取输出数据中概率最高的那一个
        _, predicted = torch.max(outputs.data, 1)
        #总共数据行
        total += labels.size(0)
        #正确的数据行
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")